# Hipercor

#### Carrefour no da resultados, así que necesitamos un plan H


In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from lxml import html
import re

Repetimos el proceso del scraping de día para carrefour

In [2]:
df = pd.read_csv('/Users/Natalio/Desktop/Cibeles/Ironhack/Proyecto_Final/Recursos/en.openfoodfacts.org.products.csv', sep='\t')
codes = df['code'].to_list()
barcodes = [i for i in codes if len(str(i)) == 13]

/Users/Natalio/miniconda3/envs/pfironhack_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,24,25,26,28,44,49) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
engine = create_engine('sqlite:///foodprices_hipercor.db')

['2,09']
<class 'str'>
['4,64']


In [ ]:
starttime = datetime.datetime.now()
print(starttime)
count = 0
for barcode in barcodes:
    count += 1
    print(f'Scraping barcode: {barcode} ------------------------------------------------ Scrapeo número: {count} ')
    #if barcode in sql:
    try:
        url = 'https://www.hipercor.es/supermercado/buscar/?term=' + barcode
        html = requests.get(url).content
        bs = BeautifulSoup(html, 'lxml')
        print('Recovering HTML')

        try:
            searchingprice = bs.find_all('div', {'class':'prices-price _current'})
            price = str(re.findall('.+(\d+.+\,.+\d+).+\€', str(searchingprice))[0])
            price = price.replace('</span>', '').replace('<span>', '')
            
            searchingpricerperkg = bs.find_all('div', {'class':'prices-price _pum'})
            pricekg = str(re.findall('.*(\d+\,\d+)\s\€', str(searchingpricerperkg))[0])
    
        except:
            print('Could not find the codebar')
            price = ['null']
            pricekg = ['null']

        finally:    
            result = list(zip([barcode], price, pricekg))
            df = pd.DataFrame(result, columns = ['code','price_hipercor','pricekg_hipercor'])
            df.to_sql('foodprices_hipercor', engine, if_exists = 'append', index=False)
        
        
    except:
        print('Coult not find the HTML request')
        pass
        
    
endtime = datetime.datetime.now()
duration = endtime - starttime
print(f'El proceso finaliza {endtime}.\nLa duración total es: {duration}')
